# Calcul impôt FR

In [212]:
# estimation taux charges, à corriger par version exacte
TAUX_CHARGES = {"non-cadre" : 0.23,
                "cadre" : 0.26}

# estimation, à corriger par version exacte
DIFF_IMPOSABLE = 1.05
TAUX_IMPOTS = [
        (11498, 0.00),  # 0% jusqu'à 11 498 EUR
        (29316, 0.11),  # 11% de 11 499 à 29 316 EUR
        (83824, 0.30),  # 30% de 29 317 à 83 824 EUR
        (180294, 0.41),  # 41% de 83 825 à 180 294 EUR
        (float('inf'), 0.45)  # 45% au-delà de 180 294 EUR
    ]

In [213]:
def calcul_net(brut, statut):
    return round(brut * (1 - TAUX_CHARGES[statut]), 1)

In [214]:
def calcul_net_imposable(net):
    return round(net * DIFF_IMPOSABLE, 1)

In [215]:
def calcul_impots(revenu_imposable):
    impot_total = 0
    revenu_restant = revenu_imposable
    seuil_precedent = 0

    # Calcul de l'impôt pour chaque tranche
    for seuil, taux in TAUX_IMPOTS:
        if revenu_restant <= 0:
            break
        if revenu_restant > seuil:
            # Calcul de l'impôt sur la portion dans la tranche
            impot_total += (seuil - seuil_precedent) * taux
            revenu_restant -= (seuil - seuil_precedent)
        else:
            # Si le revenu restant est inférieur au seuil, appliquer l'impôt sur le reste
            impot_total += revenu_restant * taux
            revenu_restant = 0

        seuil_precedent = seuil

    return round(impot_total, 2)


In [219]:
net = calcul_net(62000, "cadre")
print(net)
net_imposable = calcul_net_imposable(net)
print(net_imposable)

45880.0
48174.0


In [220]:
net_EUR = net - calcul_impots(net_imposable)
net_EUR

38262.62

# Calcul équivalent JP

In [221]:
TAUX_CHANGE_JPY_EUR = 160

In [222]:
#from forex_python.converter import CurrencyRates
#c = CurrencyRates()
#taux = c.get_rate('JPY', 'EUR')
#print(f"Le taux de change JPY/EUR est : {taux}")

In [223]:
net_JPY = net_EUR * TAUX_CHANGE_JPY_EUR
net_JPY

6122019.2

In [273]:
TAUX_IMPOTS_JP = [
    (0, 0.05),
    (1_950_000, 0.10),
    (3_300_000, 0.20),
    (6_950_000, 0.23),
    (9_000_000, 0.33),
    (18_000_000, 0.40),
    (40_000_000, 0.45)
]

TAUX_RECONSTRUCTION = 0.021  # 2.1%
DEDUCTION_PERSONNELLE = 6*380_000
TAUX_CHARGES_JP = 0.14  # 14%

In [274]:
# Fonction pour calculer l'impôt total
def calcul_impot_total(revenu_brut):
    # Calcul du revenu après déduction des charges sociales
    revenu_net_imposable = revenu_brut * (1 - TAUX_CHARGES_JP)
    #revenu_net_imposable = revenu_brut
    # Calcul du revenu imposable après la déduction personnelle
    revenu_imposable = max(0, revenu_net_imposable - DEDUCTION_PERSONNELLE)
    
    impot_base = 0
    for i in range(len(TAUX_IMPOTS_JP)):
        seuil, taux = TAUX_IMPOTS_JP[i]
        if i + 1 < len(TAUX_IMPOTS_JP):
            suivant = TAUX_IMPOTS_JP[i + 1][0]
            if revenu_imposable > seuil:
                impot_base += (min(revenu_imposable, suivant) - seuil) * taux
        else:
            if revenu_imposable > seuil:
                impot_base += (revenu_imposable - seuil) * taux

    # Appliquer la surtaxe de reconstruction (2.1% de l'impôt de base)
    #impot_total = impot_base * (1 + TAUX_RECONSTRUCTION)
    impot_total = impot_base
    return round(impot_total)

# Fonction pour estimer le brut à partir du net souhaité
def trouver_brut_depuis_net(net_vise, tolérance=100):
    bas = net_vise
    haut = net_vise * 2
    while bas <= haut:
        mid = (bas + haut) / 2
        charges = mid * TAUX_CHARGES_JP
        impot = calcul_impot_total(mid)
        net_calcule = mid - charges - impot
        if abs(net_calcule - net_vise) <= tolérance:
            return round(mid)
        elif net_calcule < net_vise:
            bas = mid + 1
        else:
            haut = mid - 1
    return None

In [275]:
net_souhaite = net_JPY
brut_estime = trouver_brut_depuis_net(net_souhaite)
impot = calcul_impot_total(brut_estime)
#charges_sociales = brut_estime * TAUX_CHARGES_JP
#net_imposable = brut_estime * (1 - TAUX_CHARGES_JP) - DEDUCTION_PERSONNELLE

print(f"✅ Net souhaité        : ¥{net_souhaite:,}")
print(f"💼 Brut estimé         : ¥{brut_estime:,}")

✅ Net souhaité        : ¥7,338,579.2
💼 Brut estimé         : ¥9,382,398


# Calcul (provisoire)

In [291]:
salaire = 82240
net = calcul_net(salaire, "cadre")
net_imposable = calcul_net_imposable(net)
net_EUR = net - calcul_impots(net_imposable)
net_JPY = net_EUR * TAUX_CHANGE_JPY_EUR
brut_estime = trouver_brut_depuis_net(net_JPY)

print(f"✅ Salaire brut FR        : €{salaire:,}")
print(f"✅ Salaire net mensuel FR : €{round(net_EUR/12):,}")
print(f"💼 Salaire net mensuel JP : ¥{round(net_JPY/12):,}")
print(f"💼 Salaire brut JP        : ¥{brut_estime:,}")

✅ Salaire brut FR        : €82,240
✅ Salaire net mensuel FR : €4,044
💼 Salaire net mensuel JP : ¥646,964
💼 Salaire brut JP        : ¥10,000,136
